In [17]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import pandas as pd

In [2]:
# Setup Selenium WebDriver
driver = webdriver.Chrome()

# URL of the page to scrape
url = 'https://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php?view=licenciamento'
driver.get(url)

In [3]:
# Wait for the button to be clickable and click at the "Filtrar" option
wait = WebDriverWait(driver, 20)
filter_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tblFilter"]/span[5]')))
# Click the button
filter_button.click()

In [18]:

# Parameters
X = 5  # Seconds to wait between attempts
Y = 10  # Maximum number of attempts
wait_box_selector = "#wait_Box"  # CSS selector for the wait box element

# Function to check if the element is displayed
def is_element_displayed(driver, selector):
    element = driver.find_element(By.CSS_SELECTOR, selector)
    return element.is_displayed()

# Initialize WebDriverWait
wait = WebDriverWait(driver, 10)  # Adjust timeout as needed

# Loop through all the options
for i in range(len(driver.find_element(By.ID, 'fc_8').find_elements(By.TAG_NAME, 'option'))):
    # Refetch the dropdown element by its ID to prevent stale element references
    dropdown = driver.find_element(By.ID, 'fc_8')

    # Create a Select object
    select = Select(dropdown)

    # Get the option at the current index
    option = select.options[i]

    # Get the option value
    option_value = option.get_attribute('value')

    # If the option has a value
    if option_value:
        # Select the option by value
        select.select_by_value(option_value)
        print(f"Selected option: {option_value}")

        # Wait for the "Atualizar" button to be clickable before clicking it
        try:
            atualizar_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tblFilter"]/span[4]')))
            # Click the "Atualizar" button
            atualizar_button.click()

            # Wait for page load (adjust time if necessary)
            time.sleep(3)  # Simple delay, adjust according to page load time

        except TimeoutException:
            print(f"Timeout while waiting for the 'Atualizar' button after selecting {option_value}")
            continue  # Skip to the next option if there is a timeout

        # Refetch the download button after clicking the "Atualizar" button to avoid stale element references
        try:
            # Wait for the "Download CSV" button to be clickable
            download_csv_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="download_csv"]')))
            # Click the "Download CSV" button
            download_csv_button.click()

            # Loop to check if the wait box is visible
            attempts = 0
            while attempts < Y:
                if is_element_displayed(driver, wait_box_selector):
                    print(f"Attempt {attempts + 1}: Wait box is still visible. Checking again in {X} seconds...")
                    time.sleep(X)
                    attempts += 1
                else:
                    print("Wait box is no longer visible.")
                    break
            else:
                print("Wait box is still visible after maximum attempts. Performing fallback action.")
                driver.refresh()

            print(f"CSV downloaded for option: {option_value}")

        except StaleElementReferenceException:
            print(f"Stale element detected. Refetching 'Download CSV' button.")
            # Retry to locate and click the download button if it's stale
            try:
                download_csv_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="download_csv"]')))
                download_csv_button.click()
            except TimeoutException:
                print(f"Timeout while refetching the 'Download CSV' button after stale reference.")
                continue
        except TimeoutException:
            print(f"Timeout while waiting for the 'Download CSV' button after selecting {option_value}")
            continue  # Skip to the next option if there is a timeout


Selected option: AC
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: AC
Selected option: AL
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: AL
Selected option: AM
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: AM
Selected option: AP
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: AP
Selected option: BA
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: BA
Selected option: CE
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is no longer visible.
CSV downloaded for option: CE
Selected option: DF
Attempt 1: Wait box is still visible. Checking again in 5 seconds...
Wait box is

In [19]:

# Close the browser after all downloads
driver.quit()